In [31]:
def getNeo4jCredentials():
    return {
        "host": "bolt://184.72.205.146:7687",
        "user": "neo4j",
        "password": "importance-eraser-characters"
    }


In [32]:
import dask.dataframe as dd
columnTypes = {
    'case:IDofConceptCase': 'string',
    'case:Includes_subCases': 'string',
    'case:Responsible_actor': 'string',
    'case:caseProcedure': 'string',
    'dateStop': 'string'
}
df = dd.read_csv('BPIC15_1.csv', dtype=columnTypes)


In [33]:
def setLinks(row):
    row['predecessor'] = row['activityNameEN'].shift(1); #lag(1)
    row['successor'] = row['activityNameEN'].shift(-1); #lead(1)
    return row;

In [34]:
df = df.sort_values(by='time:timestamp').groupby('case:concept:name').apply(setLinks)

/tmp/ipykernel_12/2874264761.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df = df.sort_values(by='time:timestamp').groupby('case:concept:name').apply(setLinks)


In [35]:
creds = getNeo4jCredentials()
driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))

with driver.session() as session:
    df['activityNameEN'].unique().apply(lambda activityName: session.write_transaction(addActivity, activityName), meta=('activityNameEN', 'object'))
session.close()